In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sqlite3

# Preprocessing
#from tensorflow import keras
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler

# Model
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
import optuna
import shap


In [2]:
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data')


# 1. Import processed data

In [3]:
df_data = pd.read_csv(os.path.join(DATA_PATH,'processed_entsoe_data.csv'))

df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86752 entries, 0 to 86751
Data columns (total 93 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   datetime                             86752 non-null  object 
 1   BE_Biomass                           86712 non-null  float64
 2   DE_Biomass                           84576 non-null  float64
 3   ES_Biomass                           86689 non-null  float64
 4   FR_Biomass                           86681 non-null  float64
 5   IT_Biomass                           86662 non-null  float64
 6   BE_Fossil Brown coal/Lignite         108 non-null    float64
 7   DE_Fossil Brown coal/Lignite         84576 non-null  float64
 8   ES_Fossil Brown coal/Lignite         86689 non-null  float64
 9   BE_Fossil Gas                        86712 non-null  float64
 10  CH_Fossil Gas                        6143 non-null   float64
 11  DE_Fossil Gas               

## Feature engineering

In [4]:
DAY = 60 * 60 * 24
YEAR = 365.2425 * DAY

def feature_engineering(df):

    df_preproc = df.copy()

    df_preproc.index = pd.to_datetime(df_preproc['datetime'], utc=True).dt.tz_convert('Europe/Paris')

    df_preproc.drop(columns=['datetime'], inplace=True)

    df_preproc['Seconds'] = df_preproc.index.map(pd.Timestamp.timestamp)

    df_preproc['Day_sin'] = np.sin(df_preproc['Seconds'] * (2 * np.pi / DAY))
    df_preproc['Day_cos'] = np.cos(df_preproc['Seconds'] * (2 * np.pi / DAY))
    df_preproc['Year_sin'] = np.sin(df_preproc['Seconds'] * (2 * np.pi / YEAR))
    df_preproc['Year_cos'] = np.cos(df_preproc['Seconds'] * (2 * np.pi / YEAR))

    df_preproc.drop(columns=['Seconds'], inplace=True)

    return df_preproc

df_preproc = feature_engineering(df_data)

## Split train test

In [7]:
COUNTRY_DICT = {
        'France': 'FR',
        'Germany': 'DE',
        'Germany_Luxembourg': 'DE',
        'Germany_Austria_Luxembourg': 'DE',
        'Italy': 'IT',
        'Italy_North': 'IT',
        'Spain': 'ES',
        'Great Britain': 'GB',
        'Netherlands': 'NL',
        'Belgium': 'BE',
        'Switzerland': 'CH',
        'Austria': 'AT',
    }

In [16]:
df_data

,datetime,BE_Biomass,DE_Biomass,ES_Biomass,FR_Biomass,IT_Biomass,BE_Fossil Brown coal/Lignite,DE_Fossil Brown coal/Lignite,ES_Fossil Brown coal/Lignite,BE_Fossil Gas,...,BE_price,CH_price,DE_price,ES_price,FR_price,IT_price,Year,Month,Day,Hour
0,2015-01-01 00:00:00+01:00,68.0,5014.50,447.0,193.00,NaN,0.0,15687.25,329.0,2536.0,...,NaN,44.94,NaN,50.10,NaN,NaN,2015,1,1,0
1,2015-01-01 01:00:00+01:00,69.0,4966.25,449.0,193.00,NaN,0.0,15321.75,328.0,2509.0,...,NaN,43.43,NaN,48.10,NaN,NaN,2015,1,1,1
2,2015-01-01 02:00:00+01:00,69.0,5001.00,448.0,193.00,NaN,0.0,14817.50,323.0,2463.0,...,NaN,38.08,NaN,47.33,NaN,NaN,2015,1,1,2
3,2015-01-01 03:00:00+01:00,68.0,5023.75,438.0,193.00,NaN,0.0,14075.00,254.0,2243.0,...,NaN,35.47,NaN,42.27,NaN,NaN,2015,1,1,3
4,2015-01-01 04:00:00+01:00,67.0,5022.75,428.0,193.00,NaN,0.0,14115.00,187.0,2134.0,...,NaN,30.83,NaN,38.41,NaN,NaN,2015,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86747,2024-12-31 19:00:00+01:00,81.0,4445.25,386.0,355.50,194.0,NaN,6731.25,0.0,807.0,...,81.30,125.02,67.77,159.44,92.78,160.00,2024,12,31,19
86748,2024-12-31 20:00:00+01:00,84.0,4238.50,387.0,355.50,195.0,NaN,5656.00,0.0,781.0,...,45.60,114.76,35.56,156.10,79.04,156.00,2024,12,31,20
86749,2024-12-31 21:00:00+01:00,86.0,4150.50,387.0,357.50,196.0,NaN,4381.00,0.0,778.0,...,17.60,110.09,15.70,150.85,50.11,143.03,2024,12,31,21
86750,2024-12-31 22:00:00+01:00,85.0,4072.00,390.0,355.25,196.0,NaN,4321.00,0.0,778.0,...,4.04,113.60,9.06,144.40,63.36,141.00,2024,12,31,22


In [26]:
country = 'ES'

country_cols = df_data.columns[df_data.columns.str.contains(country)].tolist()


# Add the column 'datetime' at the end of the list 
country_cols = country_cols + ['datetime']

country_cols

['ES_Biomass',
 'ES_Fossil Brown coal/Lignite',
 'ES_Fossil Gas',
 'ES_Fossil Hard coal',
 'ES_Fossil Oil',
 'ES_Hydro Pumped Storage_Consumption',
 'ES_Hydro Pumped Storage_Generation',
 'ES_Hydro Run-of-river and poundage',
 'ES_Hydro Water Reservoir',
 'ES_Nuclear',
 'ES_Other',
 'ES_Solar',
 'ES_Waste',
 'ES_Wind Offshore',
 'ES_Wind Onshore',
 'ES_actual_load',
 'ES_price',
 'datetime']

In [ ]:
# Looking at missing values in generation data
country_missing_index = {}

for country in list(set(COUNTRY_DICT.values())):
    #list all columns for the country

    print(country)

    country_cols = df_data.columns[df_data.columns.str.contains(country)].tolist()

    if country_cols == []:
        next

    else:
        
        df_country = df_data[country_cols].copy()

        df_country.drop(columns=[country + '_price', country + '_actual_load'], inplace=True)

        country_missing_index[country] = df_country[df_country.isnull().all(axis=1)].index.tolist()

        print(country_missing_index[country])
    
    

ES
[451, 452, 643, 661, 971, 1017, 2136, 2505, 2685, 2859, 2868, 2869, 2870, 2871, 2872, 2873, 3945, 4273, 6410, 8547, 10684, 12821, 14958, 17095, 19232, 21369, 23506, 25643, 27780, 29917, 32054, 33408, 34191, 36328, 38465, 40602, 42739, 43175, 43176, 43177, 43178, 43179, 44876, 46688, 47013, 49150, 51287, 53424, 55561, 57698, 59835, 61972, 64109, 66246, 68383, 70520, 72657, 74794, 76931, 79068, 81205, 83342, 85479]
IT
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 2136, 4273, 6410, 8547, 10684, 12821, 14958, 17095, 19232, 21369, 23506, 25643, 27780, 29917, 32054, 34191, 36328, 38465, 40602, 42739, 44876, 47013, 49150, 51287, 53424, 55561, 57698, 59835, 61972, 64109, 66246, 68383, 70520, 72657, 74794, 76931, 79068, 81205, 83342, 85479]
AT
NL
FR
[2136, 4273, 6034, 6410, 8547, 8949, 9207, 10684, 12821, 12930, 13651, 13745, 13746, 14921, 14958, 15235, 15236, 15239, 15830, 17095, 19232, 21369, 23506, 25643, 27780, 28029, 29917, 32054, 34191, 36328, 3

# 2. Forecast with XGBoost